In [ ]:
import pickle as pkl
import os
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
with open('/home/student/Desktop/Groundeep/batched_train_data_from_mat.pkl', 'rb') as f:
    data = pkl.load(f)


binary_data = np.where(data['data'] >= 0.5, 255, 0).astype(np.uint8)

# Salvataggio come file .npz
np.savez('/home/student/Desktop/Groundeep/circle_dataset_100x100/binary_de_wind.npz', binary_data=binary_data)


In [5]:
data = np.load('/home/student/Desktop/Groundeep/circle_dataset_100x100/binary_de_wind.npz')

data = data['binary_data']

data.shape

(659, 100, 10000)